## Como limpiar tus datos en Python

Todo lo explicado en este notebook pertenece al artículo How to Clean Your Data in Python de Huong Ngo, por lo cual anexo el enlace a este mismo. Adem&aacute;s tambi&eacute;n adjunto el enlace de donde sali&oacute; el conjunto de datos.
<p>Click para ver el art&iacute;culo <a href="https://towardsdatascience.com/how-to-clean-your-data-in-python-8f178638b98d">How to Clean your Data</a>.</p>
<p>Click para descargar el conjunto de datos <a href="https://www.kaggle.com/datasets/shivamb/netflix-shows">Netflix Movies and TV Shows Dataset</a>.</p>

El artículo es creado como punto de partida para aprender a limpiar los datos de forma eficiente para así utilizarlo en proyectos personales. Además, se trabajará con el conjunto de datos de películas y programas de televisión en Netflix, el cual presenta ciertas inconsistencias y datos faltantes.

Primeramente importamos las librerías necesarias para trabajar en este conjunto de datos.

In [1]:
import pandas as pd
from datetime import datetime


Paso 1. Investigar los datos


Antes de realizar cualquier acción en el conjunto de datos, primero tenemos que revisar los datos para poder comprender con que variables se está trabajando, como se estructuran estos valores según la columna en la que se encuentran, para así tener una idea aproximada de lo que se necesita abordar o podrían ocasionar problemas en la fase de análisis. También se podrán eliminar ciertas columnas que no son necesarias, dependiendo del análisis que se quiere realizar.

1.1 Imprimir las primeras filas del conjunto de datos

Se imprimen las primeras 5 filas del conjunto de datos, se recomienda imprimir menos de 10 para no abrumarse al momento de analizar rápido el conjunto de datos.

In [2]:
# Importando el conjunto de datos
netflix_titles = pd.read_csv("netflix_titles.csv")
# Se imprimen las primeras 5 columnas del conjunto de datos
netflix_titles.head()

show_id     type                  title         director  \
0      s1    Movie   Dick Johnson Is Dead  Kirsten Johnson   
1      s2  TV Show          Blood & Water              NaN   
2      s3  TV Show              Ganglands  Julien Leclercq   
3      s4  TV Show  Jailbirds New Orleans              NaN   
4      s5  TV Show           Kota Factory              NaN   

                                                cast        country  \
0                                                NaN  United States   
1  Ama Qamata, Khosi Ngema, Gail Mabalane, Thaban...   South Africa   
2  Sami Bouajila, Tracy Gotoas, Samuel Jouy, Nabi...            NaN   
3                                                NaN            NaN   
4  Mayur More, Jitendra Kumar, Ranjan Raj, Alam K...          India   

           date_added  release_year rating   duration  \
0  September 25, 2021          2020  PG-13     90 min   
1  September 24, 2021          2021  TV-MA  2 Seasons   
2  September 24, 2021          2021  TV-MA   1 Season   
3  September 24, 2021          2021  TV-MA   1 Season   
4  September 24, 2021          2021  TV-MA  2 Seasons   

                                           listed_in  \
0                                      Documentaries   
1    International TV Shows, TV Dramas, TV Mysteries   
2  Crime TV Shows, International TV Shows, TV Act...   
3                             Docuseries, Reality TV   
4  International TV Shows, Romantic TV Shows, TV ...   

                                         description  
0  As her father nears the end of his life, filmm...  
1  After crossing paths at a party, a Cape Town t...  
2  To protect his family from a powerful drug lor...  
3  Feuds, flirtations and toilet talk go down amo...  
4  In a city of coaching centers known to train I...

Con esto se tendrá una buena idea de con que tipos de datos se estará tratando, que columnas requieren limpieza o transformaciones y que otros datos se pueden extraer.

1.2 Guardar las variables en una lista

Esto se hace con el fin de tener un fácil acceso a las diferentes columnas del conjunto de datos, principalmente cuando se quieren hacer las mismas transformaciones en diferentes subconjuntos de columnas.

In [3]:
# Trayendo las columnas del conjunto de datos
columns = list(netflix_titles.columns)
columns

['show_id',
 'type',
 'title',
 'director',
 'cast',
 'country',
 'date_added',
 'release_year',
 'rating',
 'duration',
 'listed_in',
 'description']

1.3 Anota los posibles problemas que se tendrán que abordar en cada columna.

Para mantenerse organizado, hay que observar los problemas que ve en su subconjunto de datos, algunas cosas a notar son:

- Hay algunas columnas con datos faltantes.
- Hay columnas con palabras y números, como date_added y duration.
- Hay 2 columnas con varias palabras distintas unidas por una coma.
- Es posible que a otras columnas les falten valores.

Paso 2. Observar la proporcion de datos faltantes.

In [4]:
# examining missing values
print("Missing values distribution: ")
print(netflix_titles.isnull().mean())
print("")

Missing values distribution: 
show_id         0.000000
type            0.000000
title           0.000000
director        0.299080
cast            0.093675
country         0.094357
date_added      0.001135
release_year    0.000000
rating          0.000454
duration        0.000341
listed_in       0.000000
description     0.000000
dtype: float64



Por este fragmento de código podemos observar la distribución de los valores faltantes en el conjunto de datos, para así tener una buena idea de en que columnas se necesitara trabajar para resolver el problema de los valores faltantes.

A partir del resultado, esto es lo que se puede recopilar:

- La columna director tiene el porcentaje más alto de datos faltantes con un 30%
- La columna cast y country tienen un porcentaje considerable de datos faltantes con un 9%
- A date_added, rating y duration le faltan únicamente 0%-0,1%
- La mayoría de las demás columnas no están vacías.

Entonces, ¿cómo se tratan estas columnas con valores faltantes?

Hay varias opciones:

- Eliminar la columna. Si no es importante para el análisis, eliminala.
- Conserva la columna. En este caso, las columnas director, cast y country son muy importantes para el análisis, así que se conservan.
- Imputación. Reemplazar datos faltantes con valores sustituidos, lo cual en este caso no es posible porque la mayoría de los datos son cadenas y no valores numéricos.

Paso 3. Verificar el tipo de datos de cada cadena

In [5]:
# Verificando el tipo de dato de cada columna
print("Column datatypes: ")
print(netflix_titles.dtypes)

Column datatypes: 
show_id         object
type            object
title           object
director        object
cast            object
country         object
date_added      object
release_year     int64
rating          object
duration        object
listed_in       object
description     object
dtype: object


Se puede ver que todas las columnas tienen como tipo de dato Object, además de release_year. En pandas, Object significa un dato de cadena o mixto (tipo numéricos y no numérico). A partir de nuestro conjunto de datos, se podrá saber que columnas son únicamente de cadena y cuáles de tipo mixto.

Paso 4. Si hay columnas de tipo cadena, verificar si hay espacios en blanco al final.

Una vez sepamos con qué tipo de datos tratamos, hay que asegurarse de eliminar los caracteres finales y los espacios en blanco utilizando strip.

In [6]:
# Trayendo todas las columinas con valores del tipo cadena/mixtos
str_cols = list(netflix_titles.columns)
str_cols.remove('release_year')

# Eliminación de los caracteres iniciales y finales de las columnas de tipo str
for i in str_cols:
    netflix_titles[i] = netflix_titles[i].str.strip()

Paso 5. Tratar con los valores faltantes (valores NaN)

Las columnas con datos faltantes son director, cast, country, date_added, rating y duration. Con esto se puede segmentar las columnas segun sea el tipo cadena o mixtas.


- Cadena: director, cast, country, rating
- Mixto: date_added y duration.

NaN no es un número en pandas, es un valor de punto flotante especial que es distinto al NoneType de Python. NaN puede resultar molesto al trabajar con el, especialmente cuando se desea filtrar para gráficos o análisis.

Para valores de tipo cadena, se pueden reemplazar los valores NaN con " ", "None" o cualquier cadena que pueda indicar que no hay ningún valor en esa entrada. Así que se reemplazó " " usando la función fill, como no es una función local se reasignó los valores modificados a la columna del conjunto de datos.

In [7]:
# Nombre de las columnas
columns = ['director', 'cast', 'country', 'rating', 'date_added']

# Recorrer las columnas para rellenar las entradas con valores NaN con ""
for column in columns:
    netflix_titles[column] = netflix_titles[column].fillna("")

Paso 6. Ver si hay otras variables que se puedan obtener extrayéndolas de otras variables.

Para los valores del tipo mixto, veamos si se puede extraer algún dato para enriquecer el análisis o facilitar el proceso.

Observando la variable date_added, se puede ver que contiene el mes, fecha y año en que se agregó la película/programa. Lo mejor que se puede hacer es separarla en otras 2 columnas, además de filtrar las filas con valores NaN y reemplazarla con 0.

In [8]:
# Examinando filas con valores nulos de date_added column
rows = []
for i in range(len(netflix_titles)):
    if netflix_titles['date_added'].iloc[i] == "":
        rows.append(i)
    
# Examinar esas filas para confirmar el estado nulo
netflix_titles.loc[rows, :]
# Extracción de meses y años añadidos
month_added = []
year_added = []
for i in range(len(netflix_titles)):
    # Reemplazando valores NaN con 0
    if i in rows:
        month_added.append(0)
        year_added.append(0)
    else:
        date = netflix_titles['date_added'].iloc[i].split(" ")
        month_added.append(date[0])
        year_added.append(int(date[2]))
        
# Tornando el nombre de los meses en numeros de mes
for i, month in enumerate(month_added):
    if month != 0:
        datetime_obj = datetime.strptime(month, "%B")
        month_number = datetime_obj.month
        month_added[i] = month_number
        
# Checando todos los meses
print(set(month_added))
print(set(year_added))

# Insertando columnas del mes y año en el conjunto de datos
netflix_titles.insert(7, "month_added", month_added, allow_duplicates = True)
netflix_titles.insert(8, "year_added", year_added, allow_duplicates = True)
netflix_titles.head()


{0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12}
{2016, 2017, 2018, 2019, 2020, 2021, 0, 2008, 2009, 2010, 2011, 2012, 2013, 2014, 2015}


show_id     type                  title         director  \
0      s1    Movie   Dick Johnson Is Dead  Kirsten Johnson   
1      s2  TV Show          Blood & Water                    
2      s3  TV Show              Ganglands  Julien Leclercq   
3      s4  TV Show  Jailbirds New Orleans                    
4      s5  TV Show           Kota Factory                    

                                                cast        country  \
0                                                     United States   
1  Ama Qamata, Khosi Ngema, Gail Mabalane, Thaban...   South Africa   
2  Sami Bouajila, Tracy Gotoas, Samuel Jouy, Nabi...                  
3                                                                     
4  Mayur More, Jitendra Kumar, Ranjan Raj, Alam K...          India   

           date_added  month_added  year_added  release_year rating  \
0  September 25, 2021            9        2021          2020  PG-13   
1  September 24, 2021            9        2021          2021  TV-MA   
2  September 24, 2021            9        2021          2021  TV-MA   
3  September 24, 2021            9        2021          2021  TV-MA   
4  September 24, 2021            9        2021          2021  TV-MA   

    duration                                          listed_in  \
0     90 min                                      Documentaries   
1  2 Seasons    International TV Shows, TV Dramas, TV Mysteries   
2   1 Season  Crime TV Shows, International TV Shows, TV Act...   
3   1 Season                             Docuseries, Reality TV   
4  2 Seasons  International TV Shows, Romantic TV Shows, TV ...   

                                         description  
0  As her father nears the end of his life, filmm...  
1  After crossing paths at a party, a Cape Town t...  
2  To protect his family from a powerful drug lor...  
3  Feuds, flirtations and toilet talk go down amo...  
4  In a city of coaching centers known to train I...

Ahora, el nuevo conjunto de datos contiene las columnas month_added y year_added. Esto nos permitirá hacer algún análisis de tendencias más adelante.

En cuanto a duration, además de ser de tipo mixto, también hay 2 unidades de tiempo diferentes en esta columna, lo cual es un problema porque son 2 tipos diferentes de contenido que se miden de forma diferente en términos de tiempo. Por lo tanto, hacer gráficos para duration sería bastante difícil de interpretar si se mantienen como están. Se eligió abordarlo separando el tipo de contenido en 2 conjuntos de datos diferentes, por lo cual, la columna de duración será solo numérica y solo tendrá 1 tipo de unidad de tiempo.

In [9]:
# Separando el conjunto de datos original a conjunto de programas de television y peliculas en su respectivo 
# conjunto de datos.
shows = []
films = []

# Recorre el conjunto de datos para identificar las filas que son programas de televisión y películas.
for i in range(len(netflix_titles)):
    if netflix_titles['type'].iloc[i] == "TV Show":
        shows.append(i)
    else:
        films.append(i)
 
# Agrupando filas de las programas de television.
netflix_shows = netflix_titles.loc[shows, :]

# Agrupando filas de las peliculas
netflix_films = netflix_titles.loc[films, :]

# Restablecer el índice de los nuevos conjuntos de datos
netflix_shows = netflix_shows.set_index([pd.Index(range(0, len(netflix_shows)))])
netflix_films = netflix_films.set_index([pd.Index(range(0, len(netflix_films)))])


Debido a que la columna duration tiene cadenas y números, también se creara una función para extraer el número de esa columna y así insertarlo en las columnas de los 2 nuevos conjuntos de datos.

In [10]:
# Se obtiene la duración de la película o el número de temporadas de la serie
def getDuration(data):
    count = 0
    durations = []
    for value in data:
	# Rellenando los valores que faltan
        if type(value) is float:
            durations.append(0)
        else:
            values = value.split(" ")
            durations.append(int(values[0]))
    return durations
    
# Inserción de una nueva columna de tipo de duración para los espectáculos (renombrando la columna)
netflix_shows.insert(11, 'seasons', getDuration(netflix_shows['duration']))
netflix_shows = netflix_shows.drop(['duration'], axis = 1)
netflix_shows.head()

# Inserción de una nueva columna de tipo de duración para las películas (renombrando la columna)
netflix_films.insert(11, 'length', getDuration(netflix_films['duration']))
netflix_films = netflix_films.drop(['duration'], axis = 1)
netflix_films.head()

show_id   type                             title  \
0      s1  Movie              Dick Johnson Is Dead   
1      s7  Movie  My Little Pony: A New Generation   
2      s8  Movie                           Sankofa   
3     s10  Movie                      The Starling   
4     s13  Movie                      Je Suis Karl   

                        director  \
0                Kirsten Johnson   
1  Robert Cullen, José Luis Ucha   
2                   Haile Gerima   
3                 Theodore Melfi   
4            Christian Schwochow   

                                                cast  \
0                                                      
1  Vanessa Hudgens, Kimiko Glenn, James Marsden, ...   
2  Kofi Ghanaba, Oyafunmike Ogunlano, Alexandra D...   
3  Melissa McCarthy, Chris O'Dowd, Kevin Kline, T...   
4  Luna Wedler, Jannis Niewöhner, Milan Peschel, ...   

                                             country          date_added  \
0                                      United States  September 25, 2021   
1                                                     September 24, 2021   
2  United States, Ghana, Burkina Faso, United Kin...  September 24, 2021   
3                                      United States  September 24, 2021   
4                            Germany, Czech Republic  September 23, 2021   

   month_added  year_added  release_year rating  length  \
0            9        2021          2020  PG-13      90   
1            9        2021          2021     PG      91   
2            9        2021          1993  TV-MA     125   
3            9        2021          2021  PG-13     104   
4            9        2021          2021  TV-MA     127   

                                          listed_in  \
0                                     Documentaries   
1                          Children & Family Movies   
2  Dramas, Independent Movies, International Movies   
3                                  Comedies, Dramas   
4                      Dramas, International Movies   

                                         description  
0  As her father nears the end of his life, filmm...  
1  Equestria's divided. But a bright-eyed hero be...  
2  On a photo shoot in Ghana, an American model s...  
3  A woman adjusting to life after a loss contend...  
4  After most of her family is murdered in a terr...

Paso 7: Verificar los valores únicos de las columnas.

Más allá de los valores potencialmente faltantes, podría haber valores corruptos con los que pueda encontrarse una vez que realice el análisis. Para comprobar esto, se pueden buscar valores únicos para algunas de las columnas. 


Centrándonos en una lista de posibles valores únicos que podrían ser más fáciles y más importante, esto podría ayudar para futuros análisis. Echando un vistazo a los conjuntos de datos, las columnas country, rating, listed_in probablemente sean las de interés, hay que examinar primero la columna de calificación, ya que parece ser la menos complicada de tratar.

Esto se puede obtener fácilmente de los valores únicos de una columna como calificación utilizando la función incorporada de Python unique.

In [11]:
# Obtener la calificación única de las películas
netflix_films['rating'].unique()

array(['PG-13', 'PG', 'TV-MA', 'TV-PG', 'TV-14', 'TV-Y', 'R', 'TV-G',
       'TV-Y7', 'G', 'NC-17', '74 min', '84 min', '66 min', 'NR', '',
       'TV-Y7-FV', 'UR'], dtype=object)

In [12]:
# Obtención de los índices de audiencia únicos de los programas
netflix_shows['rating'].unique()

array(['TV-MA', 'TV-14', 'TV-Y7', 'TV-PG', 'TV-Y', 'TV-G', 'R', 'NR', '',
       'TV-Y7-FV'], dtype=object)

Revisando los datos de salida podemos ver que salen valores en minutos, además de existir UR (sin clasificar) y NR (No clasificado). Para investigar más a fondo, se extraerán las filas con estos datos extraños.

In [13]:
# Imprimir más detalles de las filas que tienen valoraciones incorrectas
incorrect_ratings = ['74 min', '84 min', '66 min']
for i in range(len(netflix_films)):
    if netflix_films['rating'].iloc[i] in incorrect_ratings:
        print(netflix_films.iloc[i])
        print("")

show_id                                                     s5542
type                                                        Movie
title                                             Louis C.K. 2017
director                                               Louis C.K.
cast                                                   Louis C.K.
country                                             United States
date_added                                          April 4, 2017
month_added                                                     4
year_added                                                   2017
release_year                                                 2017
rating                                                     74 min
length                                                          0
listed_in                                                  Movies
description     Louis C.K. muses on religion, eternal love, gi...
Name: 3562, dtype: object

show_id                                          

Con este fragmento de código, podemos ver que 3 filas distintas contienen esta calificación extraña y que en realidad pertenece a la columna de longitud. También podemos ver el número de fila donde se encuentra el problema, que será útil para corregir las entradas, posteriormente se procede a corregir estas entradas moviendo las "calificaciones incorrectas" (en realidad, duración) a la columna de longitud e ingresando las calificaciones correctas.

In [14]:
# Obteniendo los índices de las filas
index = [3562, 3738, 3747]

# Arreglando las entradas
for i in index:
    split_value = netflix_films['rating'].iloc[i].split(" ")
    length = split_value[0]
    netflix_films['length'].iloc[i] = length
    netflix_films['rating'].iloc[i] = "NR" 
    
    
# Se vuelve a comprobar las entradas
for i in index:
    print(netflix_films.iloc[i])

show_id                                                     s5542
type                                                        Movie
title                                             Louis C.K. 2017
director                                               Louis C.K.
cast                                                   Louis C.K.
country                                             United States
date_added                                          April 4, 2017
month_added                                                     4
year_added                                                   2017
release_year                                                 2017
rating                                                         NR
length                                                         74
listed_in                                                  Movies
description     Louis C.K. muses on religion, eternal love, gi...
Name: 3562, dtype: object
show_id                                           

C:\Users\ages0\AppData\Local\Temp\ipykernel_300\3148512868.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  netflix_films['length'].iloc[i] = length
C:\Users\ages0\AppData\Local\Temp\ipykernel_300\3148512868.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  netflix_films['rating'].iloc[i] = "NR"


Para los valores de UR y NR en la columna de calificación, se debe mantener la coherencia cuando se usa NR en el conjunto de datos netflix_shows y cambiar los valores de UR a NR.

In [15]:
# Se arreglan las entradas
for i in range(len(netflix_films)):
    if netflix_films['rating'].iloc[i] == "UR":
        netflix_films['rating'].iloc[i] = "NR"
        
# Se vuelve a verificar
netflix_films['rating'].unique()


C:\Users\ages0\AppData\Local\Temp\ipykernel_300\2767565248.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  netflix_films['rating'].iloc[i] = "NR"


array(['PG-13', 'PG', 'TV-MA', 'TV-PG', 'TV-14', 'TV-Y', 'R', 'TV-G',
       'TV-Y7', 'G', 'NC-17', 'NR', '', 'TV-Y7-FV'], dtype=object)

Ahora que se ha limpiado la columna de rating, veamos la columna país y listed_in. A estas alturas, nos damos cuenta de que no es tan fácil extraer valores únicos de la columna rating. Debido a que los valores en esas columnas son palabras unidas por comas, lo que hace más difícil extraer el conjunto de palabras y luego encontrar palabras únicas de ese conjunto.

La forma de solucionar este problema es implementando una función única para este caso especial.

La estructura de datos que conviene en este caso son los conjuntos, ya que tienen una gran capacidad para almacenar elementos únicos del mismo tipo en orden, además, para extraer aquellas palabras que están unidas por comas, se puede usar la función split para dividir la cadena por coma.

In [16]:

# Función para obtener valores únicos de una columna
def getUnique(data):
    unique_values = set()
    for value in data:
        if type(value) is float:
            unique_values.add(None)
        else:
            values = value.split(", ")
            for i in values:
                unique_values.add(i)
    return list(unique_values)

Después de usar la función, podemos obtener fácilmente los valores únicos para las columnas country y listed_in.

In [25]:
# Obtener nombres de países únicos
unique_countries = getUnique(netflix_titles['country'])
unique_countries

['',
 'United States',
 'Brazil',
 'France',
 'Cambodia',
 'Portugal',
 'Luxembourg',
 'Syria',
 'Poland,',
 'Cyprus',
 'Peru',
 'Morocco',
 'Russia',
 'Ukraine',
 'Greece',
 'Cayman Islands',
 'Japan',
 'Kenya',
 'Lebanon',
 'Nepal',
 'United Arab Emirates',
 'Ecuador',
 'Ghana',
 'Finland',
 'South Korea',
 'Iceland',
 'Switzerland',
 'United States,',
 'Iraq',
 'United Kingdom,',
 'Singapore',
 'Ireland',
 'Iran',
 'Argentina',
 'Nicaragua',
 'Venezuela',
 'Pakistan',
 'India',
 'Mexico',
 'Algeria',
 'Montenegro',
 'Germany',
 'Qatar',
 'Poland',
 'Chile',
 'Vietnam',
 'Cambodia,',
 'Puerto Rico',
 'Burkina Faso',
 'Hungary',
 'Mongolia',
 'Vatican City',
 'Colombia',
 'Somalia',
 'Uganda',
 'Taiwan',
 'Italy',
 'Turkey',
 'Norway',
 'Canada',
 'Malawi',
 'Lithuania',
 'Nigeria',
 'Senegal',
 'Egypt',
 'China',
 'Slovakia',
 'Australia',
 'Bahamas',
 'Indonesia',
 'Kazakhstan',
 'Hong Kong',
 'Botswana',
 'Cameroon',
 'United Kingdom',
 'Slovenia',
 'Denmark',
 'Panama',
 'Zimbabwe

Examinemos la lista de países únicos para ver si hay inconsistencias o errores, se puede encontrar en la lista general lo siguiente:

- Están tanto la Unión Soviética como Rusia.
- Están tanto Alemania Oriental/Occidental como Alemania

Esto se soluciona con algunas modificaciones en el conjunto de datos.

In [18]:
# Convertir la unión soviética en rusia y la alemania del este/oeste en alemania
for i in range(len(netflix_titles)):
    if type(netflix_titles['country'].iloc[i]) is not float:
        countries = netflix_titles['country'].iloc[i].split(", ")
        for j in range(len(countries)):
            if "Germany" in countries[j]:
                countries[j] = "Germany"
            elif "Soviet Union" in countries[j]:
                countries[j] = "Russia"
        netflix_titles['country'].iloc[i] = ", ".join(countries)

C:\Users\ages0\AppData\Local\Temp\ipykernel_300\4280642800.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  netflix_titles['country'].iloc[i] = ", ".join(countries)


En cuanto a la lista de géneros, podemos ver que hay algunos géneros que quizás no queramos o no necesitemos incluir. Por lo tanto, podemos eliminarlo fácilmente del conjunto de datos para que nuestro análisis sea menos confuso.

In [19]:
# Conseguir géneros cinematográficos únicos
unique_genres_films = getUnique(netflix_films['listed_in'])
unique_genres_films

['Children & Family Movies',
 'Movies',
 'Anime Features',
 'Sports Movies',
 'Classic Movies',
 'Documentaries',
 'Faith & Spirituality',
 'Dramas',
 'Music & Musicals',
 'Romantic Movies',
 'Sci-Fi & Fantasy',
 'Action & Adventure',
 'LGBTQ Movies',
 'International Movies',
 'Horror Movies',
 'Stand-Up Comedy',
 'Cult Movies',
 'Thrillers',
 'Independent Movies',
 'Comedies']

In [20]:
# Conseguir géneros de programas únicos
unique_genres_shows = getUnique(netflix_shows['listed_in'])
unique_genres_shows

['TV Dramas',
 'Anime Series',
 'TV Shows',
 'TV Thrillers',
 'TV Sci-Fi & Fantasy',
 'Stand-Up Comedy & Talk Shows',
 "Kids' TV",
 'Spanish-Language TV Shows',
 'TV Mysteries',
 'British TV Shows',
 'TV Comedies',
 'Reality TV',
 'Korean TV Shows',
 'TV Horror',
 'Science & Nature TV',
 'Docuseries',
 'Teen TV Shows',
 'Classic & Cult TV',
 'Romantic TV Shows',
 'Crime TV Shows',
 'International TV Shows',
 'TV Action & Adventure']

Tanto en el conjunto de datos de programas de televisión como de películas, hay un género de "programas de televisión" y "películas". Técnicamente, esto no es un género, pero podría ser una etiqueta del tipo de contenido. Para confirmar esto, se debe imprimir los recuentos de estos "géneros" que aparecen en los respectivos conjuntos de datos.

La hipótesis es que si estos “géneros” aparecen en todas las filas de los conjuntos de datos, significa que son simplemente etiquetas.

In [21]:
# Revisando los programas de televisión
# Reemplazando netflix_shows con netflix_films para revisar las películas
count = 0
index = []
for i, value in enumerate(netflix_shows['listed_in']):
    genres = value.split(", ")
    if "TV Shows" in genres:
        count += 1
        index.append(i)
print("count %s" %count)
print("index %s" %index)

count 16
index [59, 110, 272, 286, 452, 599, 991, 1432, 1548, 1808, 1840, 2107, 2160, 2190, 2465, 2559]


Como el recuento de "géneros" es menor que el tamaño de los conjuntos de datos, se usará la salida del código para examinar las filas.

Se escribió el código para generar específicamente los índices de filas en una lista, se puede usar fácilmente esa lista y la función iloc para obtener una vista de las filas.

In [22]:
# Impresión de las 5 primeras filas de todas las filas que tienen Programas de TV como género
netflix_shows.iloc[index[0:5]]


show_id     type                                          title  \
59     s149  TV Show                                     HQ Barbers   
110    s298  TV Show                                       Navarasa   
272    s727  TV Show                Metallica: Some Kind of Monster   
286    s772  TV Show  Pretty Guardian Sailor Moon Eternal The Movie   
452   s1332  TV Show            Five Came Back: The Reference Films   

                                              director  \
59                                     Gerhard Mostert   
110  Bejoy Nambiar, Priyadarshan, Karthik Narain, V...   
272                      Joe Berlinger, Bruce Sinofsky   
286                                         Chiaki Kon   
452                                                      

                                                  cast        country  \
59   Hakeem Kae-Kazim, Chioma Omeruah, Orukotan Ade...                  
110  Suriya, Vijay Sethupathi, Revathy, Prakash Raj...          India   
272  James Hetfield, Lars Ulrich, Kirk Hammett, Rob...  United States   
286  Kotono Mitsuishi, Hisako Kanemoto, Rina Satou,...                  
452                                                     United States   

            date_added  month_added  year_added  release_year rating  seasons  \
59   September 1, 2021            9        2021          2020  TV-14        1   
110     August 6, 2021            8        2021          2021  TV-MA        1   
272      June 13, 2021            6        2021          2014  TV-MA        1   
286       June 3, 2021            6        2021          2021  TV-14        1   
452   February 9, 2021            2        2021          1945  TV-MA        1   

    listed_in                                        description  
59   TV Shows  When a family run barber shop in the heart of ...  
110  TV Shows  From amusement to awe, the nine human emotions...  
272  TV Shows  This collection includes the acclaimed rock do...  
286  TV Shows  When a dark power enshrouds the Earth after a ...  
452  TV Shows  This collection includes 12 World War II-era p...

In [23]:
# Impresión de las 5 primeras filas de todas las filas que tienen Películas como género
netflix_films.iloc[index[0:5]]

show_id   type                          title         director  \
59     s108  Movie               A Champion Heart     David de Vos   
110    s175  Movie               Tears of the Sun    Antoine Fuqua   
272    s420  Movie  Chhota Bheem: Bheem vs Aliens    Rajiv Chilaka   
286    s439  Movie               2 Weeks in Lagos  Kathryn Fasegha   
452    s723  Movie                   Sir! No Sir!     David Zeiger   

                                                  cast        country  \
59   Mandy Grace, David de Vos, Donna Rusch, Devan ...  United States   
110  Bruce Willis, Monica Bellucci, Cole Hauser, Ea...  United States   
272  Vatsal Dubey, Julie Tejwani, Rupa Bhimani, Jig...                  
286  Beverly Naya, Mawuli Gavor, Ajoke Silva, Jide ...                  
452                                        Troy Garity  United States   

            date_added  month_added  year_added  release_year rating length  \
59   September 4, 2021            9        2021          2018      G     90   
110  September 1, 2021            9        2021          2003      R    121   
272      July 22, 2021            7        2021          2010  TV-Y7     69   
286      July 16, 2021            7        2021          2020  TV-PG    107   
452      June 15, 2021            6        2021          2005  TV-MA     84   

                                         listed_in  \
59                Children & Family Movies, Dramas   
110                     Action & Adventure, Dramas   
272        Children & Family Movies, Sports Movies   
286  Dramas, International Movies, Romantic Movies   
452                                  Documentaries   

                                           description  
59   When a grieving teen must work off her debt to...  
110  A Navy SEAL is sent to a war-torn African jung...  
272  Space invaders have kidnapped Dholakpur’s king...  
286  A businessman returns home to Nigeria and fall...  
452  This documentary chronicles the largely forgot...

Al observar las filas, ahora se puede ver que el género “Programas de TV” y “Películas” se usó para indicar que estos contenidos no tenían un género en primer lugar. Entendiendo lo que esto significa, se puede optar por excluirlo o incluirlo en el análisis, en este caso, se eligió incluirlo porque no afecta el análisis.

Aunque este paso es tedioso, también es muy importante ya que nos permite encontrar los problemas en nuestro conjunto de datos que están ocultos a primera vista.